In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report 

c:\Users\lesli\anaconda3\envs\dev\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# loading csv data to a Pandas Dataframe
disease_data = pd.read_csv('resources/disease_data.csv')

symptom_severity = pd.read_csv('resources/symptom_severity.csv')

In [3]:
# Assuming disease_data is your DataFrame
features = disease_data.drop('Disease', axis=1)  
labels = disease_data['Disease']  



In [4]:
# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [5]:
# Convert labels to one-hot encoded vectors
num_classes = len(label_encoder.classes_)
y_onehot = tf.keras.utils.to_categorical(encoded_labels, num_classes)



In [6]:
# One-hot encode the categorical features (symptoms)
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
features_onehot = onehot_encoder.fit_transform(features)

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_onehot, encoded_labels, test_size=0.2, random_state=42)


In [8]:
# Convert labels to one-hot encoded vectors using NumPy
y_train_onehot = tf.keras.utils.to_categorical(np.array(y_train), num_classes)
y_test_onehot = tf.keras.utils.to_categorical(np.array(y_test), num_classes)


In [9]:
#Assuming y_test is a 1D array of integers representing class labels
num_classes = 41
y_test_onehot = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

In [10]:
# Create a new scaler for the test set
scaler_test = MinMaxScaler()

In [11]:
#Fit the scaler on the training data
scaler_test.fit(X_train)

MinMaxScaler()

In [12]:
# Transform the test features using the same scaler
X_test_scaled = scaler_test.transform(X_test)

In [13]:
# Convert NumPy arrays to TensorFlow tensors
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train_onehot, dtype=tf.float32)
X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tf = tf.convert_to_tensor(y_test_onehot, dtype=tf.float32)



In [14]:
# Create TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tf, y_train_tf))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_tf, y_test_tf))


In [15]:
# Shuffle and batch the datasets
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [16]:
# Define a simple neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

In [17]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [18]:
# Train the model with a specified number of epochs
epochs = 100
model.fit(train_dataset, epochs=epochs, validation_data=test_dataset)

Epoch 1/100


123/123 [==============================] - 4s 13ms/step - loss: 2.5806 - accuracy: 0.6900 - val_loss: 0.9620 - val_accuracy: 0.9929
Epoch 2/100
123/123 [==============================] - 2s 11ms/step - loss: 0.2920 - accuracy: 0.9997 - val_loss: 0.0736 - val_accuracy: 1.0000
Epoch 3/100
123/123 [==============================] - 1s 9ms/step - loss: 0.0350 - accuracy: 1.0000 - val_loss: 0.0257 - val_accuracy: 1.0000
Epoch 4/100
123/123 [==============================] - 1s 8ms/step - loss: 0.0141 - accuracy: 1.0000 - val_loss: 0.0138 - val_accuracy: 1.0000
Epoch 5/100
123/123 [==============================] - 1s 7ms/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 0.0088 - val_accuracy: 1.0000
Epoch 6/100
123/123 [==============================] - 1s 8ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.0062 - val_accuracy: 1.0000
Epoch 7/100
123/123 [==============================] - 1s 8ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.0046 - val_accuracy: 1.

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_onehot, verbose=2)


31/31 - 0s - loss: 2.6481e-06 - accuracy: 1.0000 - 295ms/epoch - 10ms/step


In [20]:
# Print the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 2.6481395707378397e-06, Accuracy: 1.0


In [21]:
# Get predictions
y_pred = model.predict(X_test_scaled)

31/31 [==============================] - 0s 3ms/step


In [22]:
# Convert one-hot encoded labels back to integers
y_test_labels = np.argmax(y_test_onehot, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)

In [23]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_test_labels, y_pred_labels)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[18  0  0 ...  0  0  0]
 [ 0 20  0 ...  0  0  0]
 [ 0  0 32 ...  0  0  0]
 ...
 [ 0  0  0 ... 24  0  0]
 [ 0  0  0 ...  0 26  0]
 [ 0  0  0 ...  0  0 21]]


In [24]:
# Classification Report
class_report = classification_report(y_test_labels, y_pred_labels)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        20
           2       1.00      1.00      1.00        32
           3       1.00      1.00      1.00        29
           4       1.00      1.00      1.00        19
           5       1.00      1.00      1.00        31
           6       1.00      1.00      1.00        28
           7       1.00      1.00      1.00        30
           8       1.00      1.00      1.00        27
           9       1.00      1.00      1.00        19
          10       1.00      1.00      1.00        25
          11       1.00      1.00      1.00        22
          12       1.00      1.00      1.00        23
          13       1.00      1.00      1.00        30
          14       1.00      1.00      1.00        21
          15       1.00      1.00      1.00        25
          16       1.00      1.00      1.00        24
    